# Preprocessing Videos

This code is to process the videos into 5 minute chunks that collect meta data for each of those chunks and save them into a csv file. The metadata contains a mix of audio and visual information that gives the reader a holistic view of the events in chunks as well as the relevant contextual information surrounding them.

### Installation

In [ ]:
%pip install --upgrade --quiet google-genai gitingest

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.0/262.0 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.3/68.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 5.5 MB/s eta 0:00:00


In [ ]:
!pip install av

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 13.9 MB/s eta 0:00:00


### Setup Gemini Code

In [ ]:
# @title
import sys
import nest_asyncio
from IPython.display import Audio, Image, Markdown, Video, display
from gitingest import ingest
from google.genai.types import CreateCachedContentConfig, GenerateContentConfig, Part
import io, av
from google.genai.types import Part
import json

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

In [ ]:
import os

from google import genai

# fmt: off
PROJECT_ID = "multimodal-event-detection"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
# fmt: on
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

In [ ]:
nest_asyncio.apply()
MODEL_ID = "gemini-2.5-flash"  # @param {type: "string"}

### Utilities

In [ ]:
import os, math, json, shlex, subprocess, uuid
from IPython.display import Video, display

def _probe_duration_sec(video_path: str) -> float:
    out = subprocess.check_output(shlex.split(
        f'ffprobe -v error -show_entries format=duration -of json "{video_path}"'
    ))
    return float(json.loads(out)["format"]["duration"])

def slice_video_reencode(video_path: str, chunk_sec: int, out_dir: str):
    """
    Slice into chunk_sec segments and re-encode to H.264 yuv420p + AAC so the browser can play.
    Use -ss AFTER -i for accurate cutting; add +faststart for seekable MP4 in notebooks.
    """
    os.makedirs(out_dir, exist_ok=True)
    total = _probe_duration_sec(video_path)
    n = max(1, math.ceil(total / chunk_sec))
    results = []

    for i in range(n):
        start = i * chunk_sec
        dur   = min(chunk_sec, max(0.001, total - start))
        if dur <= 0: break

        out_path = os.path.join(out_dir, f"chunk_{i:02d}_{uuid.uuid4().hex}.mp4")
        cmd = (
            f'ffmpeg -hide_banner -loglevel error '
            f'-i "{video_path}" -ss {start:.3f} -t {dur:.3f} '      # ss AFTER -i = accurate seek
            f'-analyzeduration 100M -probesize 100M '
            f'-c:v libx264 -preset veryfast -crf 23 -pix_fmt yuv420p '
            f'-c:a aac -b:a 128k '
            f'-movflags +faststart "{out_path}"'
        )
        subprocess.check_call(shlex.split(cmd))
        results.append((i, out_path, start, start + dur))

    return results

import json, re

def parse_model_json(raw: str):
    """
    Return a Python dict from a model's 'JSON' response.
    Handles common annoyances: code fences, leading text, whitespace.
    Raises JSONDecodeError if it truly isn't valid JSON.
    """
    if raw is None:
        raise json.JSONDecodeError("Empty response", "", 0)

    s = raw.strip()

    # 1) Remove fenced code blocks if present (```json ... ``` or ``` ... ```)
    fence_re = re.compile(r"^\s*```(?:json)?\s*([\s\S]*?)\s*```\s*$", re.IGNORECASE)
    m = fence_re.match(s)
    if m:
        s = m.group(1).strip()

    # 2) If there's still stray prose, try to extract the first top-level JSON object
    #    (find first '{' and its matching closing '}' by counting braces)
    if not s.lstrip().startswith("{"):
        # find first '{'
        start = s.find("{")
        if start != -1:
            # walk forward to find the matching closing brace
            depth = 0
            end = None
            for i, ch in enumerate(s[start:], start=start):
                if ch == "{":
                    depth += 1
                elif ch == "}":
                    depth -= 1
                    if depth == 0:
                        end = i + 1
                        break
            if end:
                s = s[start:end].strip()

    # 3) Finally parse
    return json.loads(s)

import re
from typing import Any, Dict, List, Union

def shift_timestamps_in_json(
    json_obj: Union[Dict, List],
    segment_start_sec: float,
    *,
    numeric_keys: List[str] = ("start_sec", "end_sec", "time_sec"),
    shift_any_key_ending_with_sec: bool = True,
    # Optional guard to reduce double-shifting already-absolute values:
    guard_small_values: bool = False,
    max_clip_len_sec: float = 60.0
) -> Union[Dict, List]:
    """
    Shift timestamps by segment_start_sec in both:
      - String values containing mm:ss or h:mm:ss patterns
      - Numeric fields for keys like start_sec, end_sec, time_sec (or any *_sec if enabled)

    Args:
        json_obj: Parsed Gemini output (dict/list)
        segment_start_sec: Segment start time (seconds) to add
        numeric_keys: Exact keys to shift (if present and numeric)
        shift_any_key_ending_with_sec: If True, also shift any numeric field whose key endswith("_sec")
        guard_small_values: If True, only shift numeric seconds if value <= max_clip_len_sec
        max_clip_len_sec: Max seconds of the local segment to decide whether to shift when guarding

    Returns:
        Updated dict/list with shifted timestamps.
    """

    # Matches h:mm:ss or mm:ss (1–2 digits for h and mm, 2 digits for ss)
    # Examples: "0:14", "1:30", "1:02:09"
    ts_pattern = re.compile(r"\b(?:(\d{1,2}):)?([0-5]?\d):([0-5]\d)\b")

    def shift_string_ts(text: str) -> str:
        def repl(m: re.Match):
            h = m.group(1)
            mm = int(m.group(2))
            ss = int(m.group(3))
            total = (int(h) * 3600 if h is not None else 0) + mm * 60 + ss
            total += segment_start_sec
            # Render as h:mm:ss if >= 3600, else mm:ss
            if total >= 3600:
                hh = int(total // 3600)
                rem = int(total % 3600)
                new_mm = rem // 60
                new_ss = rem % 60
                return f"{hh}:{new_mm:02d}:{new_ss:02d}"
            else:
                new_mm = int(total // 60)
                new_ss = int(total % 60)
                return f"{new_mm}:{new_ss:02d}"
        return ts_pattern.sub(repl, text)

    def should_shift_numeric(key: str, val: Any) -> bool:
        if not isinstance(val, (int, float)):
            return False
        if key in numeric_keys:
            pass_ok = True
        elif shift_any_key_ending_with_sec and key.endswith("_sec"):
            pass_ok = True
        else:
            return False
        if guard_small_values:
            return val <= max_clip_len_sec
        return True

    def walk(obj: Any) -> Any:
        if isinstance(obj, dict):
            new_d = {}
            for k, v in obj.items():
                if should_shift_numeric(k, v):
                    new_d[k] = float(v) + float(segment_start_sec)
                else:
                    new_d[k] = walk(v)
            return new_d
        elif isinstance(obj, list):
            return [walk(v) for v in obj]
        elif isinstance(obj, str):
            return shift_string_ts(obj)
        else:
            return obj

    return walk(json_obj)



### Pre-processing Code

In [ ]:
LAW_ENF_SCHEMA = {
  "type": "OBJECT",
  "properties": {
    # (A) ORIGINAL PROSE FIELDS (strings)
    "speech_and_audio_cues_description": {"type": "STRING"},
    "audio_context_description": {"type": "STRING"},
    "use_of_force_description": {"type": "STRING"},
    "compliance_and_behavior_description": {"type": "STRING"},
    "excessive_force_description": {"type": "STRING"},
    "key_moments_summary": {"type": "STRING"},

    # (B) OBJECTIVE METRICS
    # Scene metrics
    "scene_type": {"type": "STRING", "enum": ["indoor","outdoor","vehicle","unknown"]},
    "time_of_day": {"type": "STRING", "enum": ["day","dusk","night","dawn","unknown"]},
    "lighting": {"type": "STRING", "enum": ["daylight","night","artificial","low_light","mixed","unknown"]},
    "weather": {"type": "STRING", "enum": ["clear","rain","snow","windy","fog","unknown"]},
    "camera_motion": {"type": "STRING", "enum": ["stable","walking","running","vehicle_moving","unknown"]},

    # Camera quality/visibility

    "camera_obfuscation_present": {"type": "BOOLEAN"},
    "camera_obfuscation_spans": {
      "type": "ARRAY",
      "items": {
        "type": "OBJECT",
        "properties": {
          "start_sec": {"type": "NUMBER"},
          "end_sec": {"type": "NUMBER"},
          "type": {"type": "STRING", "enum": ["occlusion","low_resolution","glare","darkness", "blur", "other"]}
        },
        "required": ["start_sec","end_sec"]
      }
    },

    # People counts / languages / ethnicities
    "officers_count": {"type": "INTEGER"},
    "officer_ethnicities": {
      "type": "ARRAY",
      "items": {"type": "STRING", "enum": [
        "white","black","hispanic","asian","native_american","pacific_islander","middle_eastern","mixed","unknown"
      ]}
    },
    "civilians_count": {"type": "INTEGER"},
    "civilian_ethnicities": {
      "type": "ARRAY",
      "items": {"type": "STRING", "enum": [
        "white","black","hispanic","asian","native_american","pacific_islander","middle_eastern","mixed","unknown"
      ]}
    },
    "languages": {"type": "ARRAY", "items": {"type": "STRING"}},  # e.g., ["en","es"]

    # Use-of-force (objective)
    "use_of_force_present": {"type": "BOOLEAN"},
    "use_of_force_types": {
      "type": "ARRAY",
      "items": {"type": "STRING", "enum": [
        "physical_contact","takedown","baton","taser_draw","taser_deploy",
        "firearm_draw","firearm_discharge","pepper_spray",
        "restraint_handcuff","restraint_prone","other"
      ]}
    },

    # Excessive-force flag (objective)
    "potential_excessive_force_bool": {"type": "BOOLEAN"}
  },

  # Make EVERY property required
  "required": [
    "speech_and_audio_cues_description",
    "audio_context_description",
    "use_of_force_description",
    "compliance_and_behavior_description",
    "excessive_force_description",
    "key_moments_summary",

    "scene_type",
    "time_of_day",
    "lighting",
    "weather",
    "camera_motion",

    "camera_obfuscation_present",
    "camera_obfuscation_spans",

    "officers_count",
    "officer_ethnicities",
    "civilians_count",
    "civilian_ethnicities",
    "languages",

    "use_of_force_present",
    "use_of_force_types",
    "potential_excessive_force_bool"
  ]
}


In [ ]:
PROMPT = """
You are an expert multimodal analysis system reviewing law-enforcement body-camera footage.
You will receive a video segment. Your task is to analyze it objectively and return a
JSON object that **strictly conforms** to the given schema.

Follow these rules:

1. **Structure**
   - Output must be valid JSON.
   - Every key in the schema must appear exactly once.
   - Values must match their expected types (STRING, BOOLEAN, INTEGER, ARRAY, or OBJECT).
   - For arrays, use [] if nothing is detected.
   - For enums, choose one of the listed values or "unknown" if unsure.
   - For booleans, use true/false (lowercase).
   - For numeric values, use integers or floats as appropriate.

2. **Descriptive fields (STRING)**
   - Provide concise, factual summaries based solely on visible or audible evidence.
   - Do not infer emotion, intent, or guilt.
   - Include timestamps and all relevant details
   - Fields: `speech_and_audio_cues_description`, `audio_context_description`,
     `use_of_force_description`, `compliance_and_behavior_description`,
     `excessive_force_description`, `key_moments_summary`.

3. **Objective metrics**
   - `scene_type`, `time_of_day`, `lighting_enum`, `weather_enum`, `camera_motion_enum`:
     classify the environment.
   - `camera_obfuscation_present`: true/false based on visibility.
   - `camera_obfuscation_spans`: list timestamps in seconds when issues occur.
   - `officers_count`, `civilians_count`: numeric counts of visible individuals.
   - `officer_ethnicities` / `civilian_ethnicities`: arrays of observed ethnicities.
   - `languages`: array of spoken or heard languages (e.g., ["en","es"]).
   - `use_of_force_present`: true if any physical contact or weapon use occurs.
   - `use_of_force_types`: list all detected force categories.
   - `potential_excessive_force_bool`: true only if the clip contains potential overuse (e.g., force after compliance).

4. **If information is NOT observable**
   - For enums: "unknown"
   - For booleans: false
   - For integers: 0
   - For arrays: []
   - For strings: "unknown"

Return ONLY the JSON. Do not include explanations or comments outside the JSON.
"""



In [ ]:
def process_gemini(video_path: str) -> dict:
    with open(video_path, "rb") as f:
        video_bytes = f.read()

    contents = [
        PROMPT,
        Part.from_bytes(data=video_bytes, mime_type="video/mp4"),
    ]

    response = client.models.generate_content(
        model=MODEL_ID,
        contents=contents,
        config={
            # Forces JSON output that conforms to the schema
            "response_mime_type": "application/json",
            "response_schema": LAW_ENF_SCHEMA
        }
    )

    raw = response.text.strip()
    try:
        parsed = json.loads(raw)
    except json.JSONDecodeError:
        # Extremely defensive: fallback to a dict of unknowns if something goes wrong
        print("ERROR")

    # Ensure all required keys exist (in case the model omitted any)
    for k in LAW_ENF_SCHEMA["required"]:
        parsed.setdefault(k, None)

    return parsed

### Testing

In [ ]:
import os
import pandas as pd

# --- Config ---
VIDEO_DIR = "/content/"    # folder containing your videos
CHUNK_MIN = 5
CHUNK_SEC = CHUNK_MIN * 60
OUT_DIR   = "/content/chunks"

os.makedirs(OUT_DIR, exist_ok=True)

# --- Helper: build summary string ---
def build_summary(data):
    parts = []
    for k, v in data.items():
        if k not in ("start_sec", "end_sec", "segment_idx", "summary"):
            parts.append(f"{k}: {v}")
    return "\n".join(parts)

# --- Process all videos in directory ---
for file in os.listdir(VIDEO_DIR):
    if not file.lower().endswith((".mp4", ".mov", ".mkv", ".avi")):
        continue

    video_path = os.path.join(VIDEO_DIR, file)
    base_name = os.path.splitext(os.path.basename(file))[0]
    csv_path = os.path.join(OUT_DIR, f"{base_name}.csv")

    # ✅ Skip if CSV already exists
    if os.path.exists(csv_path):
        print(f"⚙️  Skipping {base_name} — already processed ({csv_path})")
        continue

    print(f"\n🎥 Processing {base_name} ...")

    # Slice the video into 5-min (configurable) chunks
    slices = slice_video_reencode(video_path, CHUNK_SEC, OUT_DIR)
    print(f"Sliced into {len(slices)} chunk(s) of ~{CHUNK_MIN} min each.\n")

    rows = []

    for idx, path, start, end in slices:
        print(f"Chunk {idx} [{start:.1f}s → {end:.1f}s] → {path}")

        try:
            # 1. Run Gemini on this segment
            data = process_gemini(path)

            # 2. Parse JSON output
            # data = parse_model_json(output)

            # 3. Adjust timestamps based on segment start
            data = shift_timestamps_in_json(data, start)

            # 4. Add metadata
            data["segment_idx"] = idx
            data["start_sec"] = start
            data["end_sec"] = end

            # 5. Create human-readable summary
            data["summary"] = build_summary(data)

            rows.append(data)
        except Exception as e:
            print(f"❌ Error processing chunk {idx}: {e}")

    # Combine all chunks into a DataFrame
    df = pd.DataFrame(rows)

    # Save to CSV named after the video
    df.to_csv(csv_path, index=False)
    print(f"✅ Saved analysis → {csv_path}\n")

print("\n✅ All videos processed.")


2025-12-05 21:04:34.800 | INFO     | logging:callHandlers:1762 | NumExpr defaulting to 2 threads.



🎥 Processing 17-183-0765 BWC_Redacted-2 ...


2025-12-05 21:04:47.592 | INFO     | logging:callHandlers:1762 | AFC is enabled with max remote calls: 10.


Sliced into 1 chunk(s) of ~5 min each.

Chunk 0 [0.0s → 12.7s] → /content/chunks/chunk_00_dfce69b0bd41429caea44df1d53b5f72.mp4


2025-12-05 21:05:02.468 | INFO     | logging:callHandlers:1762 | HTTP Request: POST https://us-central1-aiplatform.googleapis.com/v1beta1/projects/multimodal-event-detection/locations/us-central1/publishers/google/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"


✅ Saved analysis → /content/chunks/17-183-0765 BWC_Redacted-2.csv


✅ All videos processed.


### Download the csv files

In [ ]:
import shutil
from google.colab import files
import os

OUT_DIR = "/content/chunks"
ZIP_PATH = "/content/chunks_output.zip"

# 1️⃣ Remove all .mp4 files
!find {OUT_DIR} -type f -name "*.mp4" -delete
print("🗑️  All .mp4 files removed from", OUT_DIR)

# 2️⃣ Zip the remaining files
shutil.make_archive(ZIP_PATH.replace(".zip", ""), 'zip', OUT_DIR)
print(f"📦 Created ZIP: {ZIP_PATH}")

# 3️⃣ Trigger download
files.download(ZIP_PATH)


🗑️  All .mp4 files removed from /content/chunks
📦 Created ZIP: /content/chunks_output.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>